# Data Masters Case: Modelo de Classificação

Felipe Viacava -- São Paulo, ago/2023

O presente documento consiste no treinamento e avaliação de modelos de classificação como parte da solução do Case "Data Masters - Cientista de Dados" do Santander Brasil. O objetivo é treinar um modelo de classificação de clientes possivelmente insatisfeitos com o banco para selecionar o público alvo de uma campanha de retenção.

Além das premissas estabelecidas na etapa de exploração dos dados, assumimos que:
- Buscamos um modelo de alto poder preditivo e não um modelo para inferência;
- Para cada cliente marcado como insatisfeito (_predicted positive_) pelo modelo, temos um custo de R$10,00;
- Destes, para os verdadeiramente insatisfeitos (_true positives_), temos um retorno de R$100,00;
- O melhor modelo é aquele que maximiza o retorno sobre a campanha de retenção, ou seja, a soma de:
    - _False Positives_, que geram custo R$10,00 por observação;
    - _True Positives_, que geram lucro de R$90,00 por observação;
    - _False Negatives_, que não geram lucro nem custo por observação;
    - _True Negatives_, que não geram lucro nem custo por observação.

As premissas acima surgem da interpretação do enunciado do problema. Outras premissas que nortearam o desenvolvimento dos modelos são:
- O código deve ser limpo, conciso e reprodutível;
- A performance dos modelos deve ser comparada sobre um conjunto de dados nunca antes visto, prevenindo _data leakage_. Este conjunto foi separado antes da análise exploratória, simulando o que seria o acompanhamento da performance de um modelo após sua implementação;
- Uso de transformadores em _pipelines_ para qualquer tipo de processamento nos dados, garantindo fácil implementação e reprodutibilidade;
- Todos os modelos testados devem obrigatoriamente passar pelos mesmos passos, sendo estes:
    - Divisão dos dados entre treino e validação, estratificado por TARGET;
    - _GridSearchCV_ para hiperparametrização maximizando a _AUC_ (conjunto de treino);
    - Retreino após a escolha dos melhores hiperparâmetros (conjunto de treino);
    - Ajuste do corte de classificação maximizando a função $\frac{90tp - 10fp}{n}$ (conjunto de validação);
    - Retreino após a escolha do melhor corte de classificação e melhores hiperparâmetros (conjunto de treino + validação);
    - Avaliação e comparação no contexto de negócios (conjunto de testes).

In [1]:
TRAIN_RF = True

## Bibliotecas

In [2]:
# --- Data Exploration and Viz --- #
import pandas as pd

# --- Classification models --- #
from sklearn.ensemble import RandomForestClassifier

# --- Preprocessing --- #
from sklearn.impute import SimpleImputer
from resources.customtransformers import \
    DropConstantColumns, \
    DropDuplicateColumns, \
    AddNonZeroCount, \
    CustomSum, \
    CustomImputer, \
    AddNoneCount, \
    CustomEncoder

# --- Pipeline Building --- #
from sklearn.pipeline import Pipeline
from resources.train_evaluate import build_model

## Leitura dos dados

In [3]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

## Modelagem

### Pipeline base

Este pipeline segue os passos sugeridos durante a análise exploratória.

In [4]:
prep = Pipeline(
    steps=[
        (
            "DropConstantColumns",
            DropConstantColumns(also=["ID"])
        ),
        (
            "DropDuplicateColumns",
            DropDuplicateColumns()
        ),
        (
            "NoneZeroCountSaldo",
            AddNonZeroCount(prefix="saldo")
        ),
        (
            "SumSaldo",
            CustomSum(prefix="saldo")
        ),
        (
            "NoneZeroCountImp",
            AddNonZeroCount(prefix="imp")
        ),
        (
            "SumImp",
            CustomSum(prefix="imp")
        ),
        (
            "ImputeNanDelta",
            CustomImputer(prefix="delta", to_replace=9999999999)
        ),
        (
            "NoneCountDelta",
            AddNoneCount(prefix="delta")
        ),
        (
            "NonZeroCountDelta",
            AddNonZeroCount(prefix="delta")
        ),
        (
            "SumDelta",
            CustomSum(prefix="delta")
        ),
        (
            "NonZeroContInd",
            AddNonZeroCount(prefix="ind")
        ),
        (
            "NonZeroCountNum",
            AddNonZeroCount(prefix="num")
        ),
        (
            "SumNum",
            CustomSum(prefix="num")
        ),
        (
            "ImputeNanVar3",
            CustomImputer(prefix="var3", to_replace=-999999)
        ),
        (
            "CustomEncoderVar36",
            CustomEncoder(colname="var36")
        ),
        (
            "CustomEncoderVar21",
            CustomEncoder(colname="var21")
        )
    ]
)

### Preenchendo NaN

Diferentemente de outros algoritmos baseados em árvores de decisão do _sklearn_, o algoritmo _RandomForestClassifier_ não tem uma estratégia definida para lidar com dados faltantes. Assim, ao final do pipeline anterior, adicionamos uma última etapa que preenche os valores faltantes com a mediana da coluna. Foi escolhida a mediana pois:
- As variáveis de prefixo "Delta" (contínua) são esparsas e a média seria muito afetada por outliers;
- Assumiu-se a variável "var3" como variável de contagem (discreta), não fazendo sentido usar a média para preenchimento;
- Como o dataset é composto por muitas variáveis e estas não foram analisadas individualmente, optou-se por não ajustar as escalas das variáveis e testar apenas modelos baseados em árvores. Assim, não faria sentido utilizar o _KNNImputer_ nesta versão da solução do _case_, e pode ser um ponto a melhorar para futuras entregas -- assim como o teste de algoritmos lineares.

In [5]:
prep_nan = Pipeline(
    steps=[
        ("prep", prep),
        ("nan", SimpleImputer(strategy="median"))
    ]
)

### Random Forest

In [6]:
rf = Pipeline(
    steps=[
        ("preprocessor", prep_nan),
        ("classifier", RandomForestClassifier())
    ]
)

rf_grid = {
    "classifier__n_estimators": [100],
    "classifier__max_depth": [10, 20],
    "classifier__max_features": ["sqrt"]
}

rf_model = build_model(
    train=TRAIN_RF,
    path="models/rf.pkl",
    train_df = train,
    test_df = test,
    model = rf,
    param_grid = rf_grid,
    target = "TARGET",
    njobs = 8,
    verbose = True
)

Splitting data into train and validation sets...
Done!
Performing GridSearchCV...
Done!
Adjusting threshold based on validation set...
Done!
Fitting model on the whole dataset...
Done!


In [10]:
rf_model.best_model_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('prep',
                                  Pipeline(steps=[('DropConstantColumns',
                                                   DropConstantColumns(also=['ID'])),
                                                  ('DropDuplicateColumns',
                                                   DropDuplicateColumns()),
                                                  ('NoneZeroCountSaldo',
                                                   AddNonZeroCount(prefix='saldo')),
                                                  ('SumSaldo',
                                                   CustomSum(prefix='saldo')),
                                                  ('NoneZeroCountImp',
                                                   AddNonZeroCount(prefix='imp')),
                                                  ('SumImp',
                                                   CustomSum(prefix='...
                                                  ('NonZeroCountNum',
                                                   AddNonZeroCount(prefix='num')),
                                                  ('SumNum',
                                                   CustomSum(prefix='num')),
                                                  ('ImputeNanVar3',
                                                   CustomImputer(prefix='var3',
                                                                 to_replace=-999999)),
                                                  ('CustomEncoderVar36',
                                                   CustomEncoder(colname='var36')),
                                                  ('CustomEncoderVar21',
                                                   CustomEncoder(colname='var21'))])),
                                 ('nan', SimpleImputer(strategy='median'))])),
                ('classifier', RandomForestClassifier(max_depth=20))])

In [8]:
rf_model.business_metrics

{'Profit (Total)': 14150,
 'Profit (per Customer)': 0.9306761378584583,
 'False Negative Loss (Total)': 30060,
 'False Negative Loss (per Customer)': 1.9771112865035516,
 'False Positive Loss (Total)': 15910,
 'False Positive Loss (per Customer)': 1.0464351486450933}

In [9]:
rf_model.classification_metrics

{'Accuracy': 0.8777295448566167,
 'Precision': 0.1735064935064935,
 'Recall': 0.5548172757475083,
 'F1': 0.26434507320933914,
 'ROC AUC': 0.828402198579269,
 'Classification Threshold': 0.09911002538370728}